In [1]:
import os
import json
import jsonlines

import torch
import ctypes
import gc
import random

In [3]:
def clean_memory():
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()
clean_memory()

In [ ]:
# metadata
# EN
# len: 4459532
# key per line: dict_keys(['meta', 'text', 'input', 'output', 'instruction'])
# 'meta' key: dict_keys(['Dataset', 'Gen', 'IFT', 'Lang', 'CFT-MR', 'CFT-P', 'CFT-SR', 'Task', 'original_path'])

In [ ]:
# {'ShareGPT', 'belle_cn', 'instinwild', 'Chinese-medical', 'alpacaGPT4', 'COIG', 'HC3', 'MOSS'}

In [17]:
import jsonlines

data_stat_path = "/home/chenxiaojia/data/competition/competition_kit/data-juicer/outputs/v6_text/en_processed_stats.jsonl"
data_path = "/home/xiejunlin/workspace/Tianchi_FT-Data_Ranker/checkpoints/run/run_keep_long_token_perplexity_refine_v6_en_2023-11-06-01-30-32/data/training_dataset.jsonl"
output = "/home/xiejunlin/workspace/Tianchi_FT-Data_Ranker/checkpoints/run/run_keep_long_token_perplexity_refine_v6_en_2023-11-06-01-30-32/data/flitered_training_dataset.jsonl"

cnt = 0
new_data_writer = jsonlines.open(output, 'w')

with jsonlines.open(data_stat_path) as file1, jsonlines.open(data_path) as file2:
    for data1, data2 in zip(file1, file2):
        # process data from both files here
        data1 = data1["__dj__stats__"]
        if data1["flagged_words_ratio"] <= 0.017:
            # print(data1["flagged_words_ratio"])
            # print(data2)
            # print("")
            cnt += 1
            new_data_writer.write(data2)

new_data_writer.close()

In [ ]:


op = CleanKeywordMapper(keywords=['http', 'www', '.com', 'href', '//'])
sample = {
    'text': 'This paper proposed a novel https://whiugc.com method on LLM',
    'target': 'This paper proposed a novel method on LLM',
}
print(op.process(sample))